In [1]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import os
import seaborn as sns
import charts_function_list
_,data,outputs = charts_function_list.folder_setup()

In [10]:
def main_pull(url,type_ = str):
    index_list = []
    url_list = []
    #set counter
    count = 0
    #
    url = url
    html = urlopen(url) #get_request
    soup = BeautifulSoup(html,"lxml")
    #pull the URL from the Pro Football Reference page you specify
    #returns the URL and something to index on
    for tr in soup.select('tbody'):
        for th in tr.find_all('th'):
            for a in th.find_all('a',href=True):
                try:
                    index_list.append(type_(a.get_text()))
                    url_list.append(base+a['href'])
   
                except:
                    pass
                    count-=1
                
                count+=1
                
                
    return count,index_list,url_list

In [ ]:
base = 'https://www.pro-football-reference.com'

#create list of current teams and the URL to their page
teams,urls = main_pull(url='https://www.pro-football-reference.com/teams/')[1:]

#pull seasons and season url, create count of seasons
season_count = []
for urls_ in urls:
    counts,season,season_url = main_pull(type_=int,url=urls_)
    season_count.append(counts)
    
#match the list to 
full_teams = [v for item,v in enumerate(teams) for x in range(season_count[item])]



stadium = []
stadium_url = []


for url in season_url:
    url = url
    html = urlopen(url) #get_request
    soup = BeautifulSoup(html,"lxml")
    
    texts = []

    for tr in soup.select('p'):
        for strong in tr.find_all('strong'):
            texts.append(strong.get_text())
            
    if (True in pd.Series(texts).isin(['Stadium:']).unique()) is False:
                stadium_url.append('')
                stadium.append('')
    else:


        for tr in soup.select('p'):
            for strong in tr.find_all('strong'):
                if strong.get_text()=='Stadium:':
                    for a in tr.find_all('a'):
                        stadium_url.append((a['href']))
                        stadium.append(a.get_text())
                else: 
                    pass

surface = []

for url in stadium_url:
    
    try:
        html = urlopen(url) #get_request
        soup = BeautifulSoup(html,"lxml")
        for tr in soup.select('p'):
            for bold in tr.find_all('b'):
                if bold.get_text()=='Surfaces:':
                    surface_text = (tr.get_text())
                    surface.append(surface_text.split(':')[1])                    
    except:
        surface.append('')
        pass
                
season_frame = pd.DataFrame({'Teams':full_teams,
                    'Season':season,
                    'Season URL':season_url,
                            'Stadium':stadium,
                            'Stadium_url':stadium_url,
                            'Surfaces':surface})

os.chdir(outputs)
season_frame.to_csv('season_frame_initial.csv')

In [138]:
season_frame = pd.read_csv('season_frame_initial.csv',index_col=0).drop('Unnamed: 0.1',axis=1)

In [47]:
#fixin surface column
surface_counts = []
for item in season_frame.drop(['Unnamed: 0'],axis=1)['Surface'].str.split(","):
    surface_counts.append(len(item))

In [109]:
def surface_appender(value):
    empty_list = []
    for item in season_frame['Surface'].str.split(","):
        fields=item
        try:
            empty_list.append(fields[value])
        except:
            empty_list.append('')
    return empty_list

In [141]:
season_frame = pd.merge(season_frame,pd.DataFrame({'Surface 1':surface_appender(0),
                     'Surface 2':surface_appender(1),
                     'Surface 3':surface_appender(2),
                     'Surface 4':surface_appender(3),
                     'Surface 5':surface_appender(4)}),left_index=True,right_index=True,how='left')

In [ ]:
#missing stadiums - to research!

season_frame[season_frame['Stadium']=='']

In [452]:
def column_split_strip(df,column):
    surface_split = df[column].str.split('-|\)|\(',expand=True).loc[:,0:2]
    surface_split[0]= surface_split[0].str.strip()
    surface_split.columns = ['Surface','Start Year','End Year']
    surface_split = surface_split.replace({'':nan})
    surface_split['End Year'] = surface_split[['End Year']].fillna(2018)
    #surface_split[['Start Year','End Year']] = surface_split[['Start Year','End Year']].astype('int')
    return surface_split
    

In [455]:
surface_1 = column_split_strip(season_frame,'Surface 1')